In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
import pickle as pkl

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1000)

In [3]:
state_list = ['Florida', 'Georgia', 'Delaware',
              'Maryland', 'Pennsylvania', 'Ohio',
              'Missouri', 'Louisiana', 'California',
              'Kansas', 'Indiana', 'New Jersey',
              'Alaska', 'Colorado', 'Tennessee',
              'District of Columbia', 'Connecticut',
              'Illinois', 'Montana', 'Mississippi',
              'Nevada', 'North Carolina', 'Massachusetts',
              'South Carolina', 'Texas', 'Virginia',
              'New York', 'Utah', 'Michigan',
              'Maine', 'Arkansas', 'Oklahoma',
              'Alabama', 'Nebraska', 'Arizona',
              'Oregon', 'New Mexico', 'Iowa',
              'New Hampshire', 'Minnesota', 'Kentucky',
              'West Virginia', 'Wisconsin', 'Washington',
              'North Dakota', 'Rhode Island', 'Idaho',
              'South Dakota', 'Hawaii', 'Vermont', 'Wyoming']

In [4]:
df = pd.read_csv('../Data Sources/gun_violence_archive_scraped_full.csv')
df = df[(df['state'].isnull()==False)].reset_index()
df.drop('index', axis = 1, inplace = True)

/Users/charlieyaris/.venvs/lpthw/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (0,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
len(df)

62581

In [6]:
raw_df = pd.read_csv('../Data Sources/gun_violence_archive_raw_html_scraped.csv').reset_index()

In [7]:
# remove pages that have been removed since scraped for a second time.
# these have been added back as new pages
remove_list = raw_df[raw_df['page_response_code']==404]['incident_id']

In [8]:
len(df)

62581

In [9]:
df = deepcopy(df[([s not in remove_list for s in df['incident_id']])])

In [10]:
len(df)

62581

In [11]:
raw_html_list = list(set(df[df['city_or_county']=='N.W. PMB #126<br />\\nWashington']['incident_id'].astype(int)))

In [12]:
print(len(set(list(raw_df['incident_id']))))
print(len(raw_html_list))

assert len(set(list(raw_df['incident_id'])))==len(raw_html_list)

646
646


In [13]:
for i, raw_html_id in enumerate(raw_df['incident_id']):
    page_id_dic = {}
    if "Geolocation" in str(raw_df.loc[i, 'raw_html']):
        try:
            page_id_dic['state'] = str(str(str(raw_df.loc[i, 'raw_html']).split('Geolocation:')[0]).split('</span>')[-2]).split(', ')[1]
        except:
            page_id_dic['state'] = None
        try:
            page_id_dic['city_or_county'] = str(str(str(str(raw_df.loc[i, 'raw_html']).split('Geolocation:')[0]).split('</span>')[-2]).split(', ')[0]).split('<span>')[1]
        except:
            page_id_dic['city_or_county'] = None
        try:
            page_id_dic['address'] = str(str(str(raw_df.loc[i, 'raw_html']).split('Geolocation:')[0]).split('</span>')[-3]).split('<span>')[-1]
        except:
            page_id_dic['address'] = None
        try:
            page_id_dic['latitude'] = str(str(raw_df.loc[i, 'raw_html']).split('Geolocation: ')[1]).split(',')[0]
        except:
            page_id_dic['latitude'] = None
        try:
            page_id_dic['longitude'] = str(str(str(raw_df.loc[i, 'raw_html']).split('Geolocation: ')[1]).split(', ')[1]).split('</span>')[0]
        except:
            page_id_dic['longitude'] = None
        possible_location_description = str(str(str(raw_df.loc[i, 'raw_html']).split('Geolocation:')[0]).split('</span>')[-4]).split('<span>')[-1]
        if possible_location_description != 'Incident':
            page_id_dic['location_description'] = possible_location_description
        else:
            page_id_dic['location_description'] = None
    else:
        try:
            page_id_dic['state'] = str(str(raw_df.loc[i, 'raw_html']).split('</span><br>\\n</div>\\n<div>\\n<h2>')[1]).split(', ')[-1]
        except:
            page_id_dic['state'] = None
        try:
            page_id_dic['city_or_county'] = str(str(str(raw_df.loc[i, 'raw_html']).split('</span><br>\\n</div>\\n<div>\\n<h2>')[1]).split(', ')[-2]).split('<span>')[-1]
        except:
            page_id_dic['city_or_county'] = None
        try:
            page_id_dic['address'] = str(str(str(str(raw_df.loc[i, 'raw_html']).split('</span><br>\\n</div>\\n<div>\\n<h2>')[1]).split(', ')[-2]).split('<span>')[-2]).split('</span>')[0]
        except:
            page_id_dic['address'] = None
        page_id_dic['latitude'] = None
        page_id_dic['longitude'] = None
        page_id_dic['location_description'] = None
    try:
        page_id_dic['congressional_district'] = str(str(raw_df.loc[i, 'raw_html']).split('Congressional District: ')[1]).split('<br>')[0]
    except:
        page_id_dic['congressional_district'] = None
    try:
        page_id_dic['state_house_district'] = str(str(raw_df.loc[i, 'raw_html']).split('State House District: ')[1]).split('<br>')[0]
    except:
        page_id_dic['state_house_district'] = None
    try:
        page_id_dic['state_senate_district'] = str(str(raw_df.loc[i, 'raw_html']).split('State Senate District: ')[1]).split('<br>')[0]
    except:
        page_id_dic['state_senate_district'] = None
        
    if page_id_dic['city_or_county']=='N.W. PMB #126<br />\\nWashington':
        try:
            page_id_dic['state'] = str(str(raw_df.loc[i, 'raw_html']).split('</span><br>\\n</div>\\n<div>\\n<h2>')[0]).split(', ')[-1]
        except:
            page_id_dic['state'] = None
        try:
            page_id_dic['city_or_county'] = str(str(str(raw_df.loc[i, 'raw_html']).split('</span><br>\\n</div>\\n<div>\\n<h2>')[0]).split(', ')[-2]).split('<span>')[-1]
        except:
            page_id_dic['city_or_county'] = None
        try:
            page_id_dic['address'] = str(str(str(str(raw_df.loc[i, 'raw_html']).split('</span><br>\\n</div>\\n<div>\\n<h2>')[0]).split(', ')[-2]).split('<span>')[-2]).split('</span>')[0]
        except:
            page_id_dic['address'] = None
        possible_location_description = str(str(str(raw_df.loc[i, 'raw_html']).split('<div class="region region-content">\\n<div id="block-system-main" class="block block-system">\\n<h1>')[1]).split('<span>')[1]).split('</span>')[0]
        if possible_location_description != page_id_dic['address']:
            page_id_dic['location_description'] = possible_location_description
        else:
            page_id_dic['location_description'] = None
    else:
        pass

#     print(i, raw_html_id)
#     print(page_id_dic)

    for key in page_id_dic.keys():
        df.loc[df['incident_id']==raw_html_id, key] = page_id_dic[key]

In [17]:
df.loc[(df['incident_id']==1276620) | (df['incident_id']==1299326), 'city_or_county'] = 'Branch (county)'
df.loc[(df['incident_id']==1276620) | (df['incident_id']==1299326), 'state'] = 'Michigan'
df.loc[df['incident_id']==1189222, 'city_or_county'] = 'Smith (county)'
df.loc[df['incident_id']==1189222, 'state'] = 'Mississippi'
df.loc[df['incident_id']==1320972, 'city_or_county'] = 'Union (county)'
df.loc[df['incident_id']==1320972, 'state'] = 'Illinois'
df.loc[df['incident_id']==1197483, 'city_or_county'] = 'Franklin (county)'
df.loc[df['incident_id']==1197483, 'state'] = 'Alabama'
df.loc[df['incident_id']==1185220, 'city_or_county'] = 'Charlotte'
df.loc[df['incident_id']==1185220, 'state'] = 'North Carolina'
df.loc[df['incident_id']==1370810, 'state'] = 'Missouri'
df.loc[df['incident_id']==1372453, 'state'] = 'Virginia'
df.loc[df['incident_id']==1359780, 'state'] = 'Texas'
df.loc[df['incident_id']==1365397, 'address'] = 'Gayosa Ave'
df.loc[df['incident_id']==1365397, 'city_or_county'] = 'Natchez'
df.loc[df['incident_id']==1365397, 'state'] = 'Mississippi'
df.loc[df['incident_id']==1365397, 'latitude'] = 31.5638
df.loc[df['incident_id']==1365397, 'longitude'] = -91.3818
df.loc[df['incident_id']==1365397, 'state_house_district'] = '94'
df.loc[df['incident_id']==1365397, 'state_senate_district'] = '38'
df.loc[df['incident_id']==1365397, 'congressional_district'] = '3'

df.loc[(df['incident_id']==1375727) | (df['incident_id']==1372647), 'address'] = '908 SE 67th St'
df.loc[(df['incident_id']==1375727) | (df['incident_id']==1372647), 'city_or_county'] = 'Oklahoma City'
df.loc[(df['incident_id']==1375727) | (df['incident_id']==1372647), 'state'] = 'Oklahoma'
df.loc[(df['incident_id']==1375727) | (df['incident_id']==1372647), 'latitude'] = 35.398389
df.loc[(df['incident_id']==1375727) | (df['incident_id']==1372647), 'longitude'] = -97.497393

df.loc[df['incident_id']==1348322, 'location_description'] = 'The Scientology Center'
df.loc[df['incident_id']==1348322, 'address'] = '315 S Market St'
df.loc[df['incident_id']==1348322, 'city_or_county'] = 'Inglewood'
df.loc[df['incident_id']==1348322, 'state'] = 'California'
df.loc[df['incident_id']==1348322, 'latitude'] = 33.960226
df.loc[df['incident_id']==1348322, 'longitude'] = -118.351936

df.loc[(['https://ktla.com/2019/03/23/driver-shoots-at-other-vehicle-on-210-freeway-in-claremont-after-tailgating-it-chp/' in s for s in df['sources'].astype(str)]), 'location_description'] = 'I-210 Freeway'
df.loc[(['https://ktla.com/2019/03/23/driver-shoots-at-other-vehicle-on-210-freeway-in-claremont-after-tailgating-it-chp/' in s for s in df['sources'].astype(str)]), 'address'] = 'I-210'
df.loc[(['https://ktla.com/2019/03/23/driver-shoots-at-other-vehicle-on-210-freeway-in-claremont-after-tailgating-it-chp/' in s for s in df['sources'].astype(str)]), 'city_or_county'] = 'Los Angeles'
df.loc[(['https://ktla.com/2019/03/23/driver-shoots-at-other-vehicle-on-210-freeway-in-claremont-after-tailgating-it-chp/' in s for s in df['sources'].astype(str)]), 'state'] = 'California'

df.loc[df['incident_id']==1374036, 'address'] = '900 Spruce St'
df.loc[df['incident_id']==1374036, 'city_or_county'] = 'Oxnard'
df.loc[df['incident_id']==1374036, 'state'] = 'California'
df.loc[df['incident_id']==1374036, 'latitude'] = 34.172615
df.loc[df['incident_id']==1374036, 'longitude'] = -119.187447

df.loc[df['incident_id']==658479, 'state'] = 'South Carolina'
df.loc[df['incident_id']==474465, 'state'] = 'Pennsylvania'
df.loc[df['incident_id']==662479, 'state'] = 'Georgia'
df.loc[df['incident_id']==460442, 'state'] = 'Arkansas'

In [18]:
num_pages = len(df[([s not in state_list for s in df['state']])])

if num_pages==0:
    print("All pages fixed.")
    print("New scraper file exported.")
    df.to_csv('../Data Sources/gun_violence_archive_scraped_full_2.csv')
else:
    print(list(df[([s not in state_list for s in df['state']])]['incident_id'].astype(int)))
    print("\n" + str(num_pages) + " pages still need to be fixed.")

All pages fixed.
New scraper file exported.
